In [1]:
import requests
import json
import spacy

Load NLP file

In [2]:
nlp = spacy.load("./Spacy_fine_tuned_NER_for_Food")

In [61]:
# text = "I had pineapple fried rice with lime based highball."
text = "I love barbecued chicken with corn salad with vanila ice cream."

queries = [] # food names from the text


# Process the text using the loaded model
doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.label_)
    queries.append(ent.text)

barbecued chicken FOOD
corn salad FOOD
ice cream FOOD


In [62]:
queries

['barbecued chicken', 'corn salad', 'ice cream']

In [63]:
app_id = "d5fd616b"
app_key = "5c06fb4f5bd545920f9d620b5a5a63d4"

Autocomplete the given query to find the food name.

In [64]:
queries_for_nutrients = [] # queries for finding the calories

for query in queries:
    body = {
        "query": query
    }

    url = "https://trackapi.nutritionix.com/v2/search/instant/?query=" + query
    headers = {
        "Content-Type": "application/json",
        "x-app-id": app_id,
        "x-app-key": app_key
    }

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Error: {response.status_code}")
    else:
        response_json = response.json()
        branded_food = 0   # check if we found the same food name as query in common food, if not, we use the first element of branded foods
        
        if "common" in response_json:
            for common in response_json["common"]:
                if common["food_name"].lower() == query.lower(): # if there's a name matching
                    branded_food = 1
                    print(common["food_name"])
                    queries_for_nutrients.append(common["food_name"])
                    break

        if branded_food == 0 and "branded" in response_json:
            if response_json["branded"]:
                print(response_json["branded"][0]["food_name"]) # pop the first element of branded food
                queries_for_nutrients.append(response_json["branded"][0]["food_name"])


Barbecued Pork Ribs
corn salad
ice cream


In [65]:
print(queries_for_nutrients)

['Barbecued Pork Ribs', 'corn salad', 'ice cream']


Since we found the food name, print the calories.

In [68]:
calories_information = [] # have {query, calories}
index = 0
total_calories = 0

url = "https://trackapi.nutritionix.com/v2/natural/nutrients"
headers = {
    "Content-Type": "application/json",
    "x-app-id": app_id,
    "x-app-key": app_key
}
for query_for_nutrients in queries_for_nutrients:
    body = {
        "query": query_for_nutrients
    }

    response = requests.post(url, headers=headers, data=json.dumps(body))

    # check if the key "foods" exists in the JSON response
    if "foods" in response.json():
        sum_calories = 0
        for food in response.json()["foods"]:
            print(food["food_name"] + ": " + str(food["nf_calories"]) + " calories")
            sum_calories += food["nf_calories"]

        total_calories += sum_calories
        calories_information.append({queries[index]: sum_calories})
        index = index + 1
    else:
        print("No foods found for query:", query)

pork ribs: 126.35 calories
corn salad: 238.7 calories
ice cream: 273.24 calories


In [69]:
print(calories_information)

[{'barbecued chicken': 126.35}, {'corn salad': 238.7}, {'ice cream': 273.24}]


In [70]:
print(total_calories)

638.29
